In [1]:
from transformer_lens import utils, HookedTransformer
from datasets import load_dataset
import torch

In [2]:
torch.set_grad_enabled(False)
DTYPE = "bfloat16"
ctx_length = 128

In [4]:
# model = HookedTransformer.from_pretrained_no_processing("gemma-2b-it", device="cuda", dtype=DTYPE)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model gemma-2b-it into HookedTransformer


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="cuda")

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
# get OWT tokens
dataset = load_dataset("stas/openwebtext-10k", split="train")
# tokenized_data = utils.tokenize_and_concatenate(data, model.tokenizer, max_length=ctx_length)
# tokenized_data = tokenized_data.shuffle(42)
# owt_tokens = tokenized_data["tokens"][:20_000].cuda()
# print("owt_tokens has shape", owt_tokens.shape)
# print("total number of tokens:", int(owt_tokens.numel()//1e6), "million")
# print()

In [12]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm


# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Create a data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Create a DataLoader
dataloader = DataLoader(tokenized_datasets, batch_size=8, collate_fn=data_collator)

# Move the model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Evaluate the model
model.eval()
total_loss = 0
num_batches = 0

with torch.no_grad():
    for batch in tqdm(dataloader):
        inputs = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['input_ids'].to(device)
        
        outputs = model(input_ids=inputs, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        num_batches += 1

        print(loss)


average_loss = total_loss / num_batches
print(f"Average loss: {average_loss}")

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

  0%|          | 1/1250 [00:00<08:30,  2.45it/s]

tensor(4.3190, device='cuda:0')


  0%|          | 2/1250 [00:00<07:15,  2.87it/s]

tensor(4.1550, device='cuda:0')


...

tensor(3.6362, device='cuda:0')


100%|█████████▉| 1249/1250 [06:21<00:00,  3.28it/s]

tensor(4.6193, device='cuda:0')


100%|██████████| 1250/1250 [06:22<00:00,  3.27it/s]

tensor(4.3805, device='cuda:0')
Average loss: 4.177739428138733


In [32]:
batch_size = 1

for b in range(10):
    batch = owt_tokens[b*batch_size:(b+1)*batch_size]
    loss = model(batch, return_type="loss", loss_per_token=True, prepend_bos=False)

    inds = loss[:, 1:].topk(1).indices[0]
    bad_tokens = batch[0, inds-2: inds+2]
    print(model.to_str_tokens(bad_tokens))

[' helpful', ' to', ' the', ' very']
[' the', ' time', '.', '\n\n']
['4', '-', 'bet', '.']
['.', ' Even', ' so', ',']
['<bos>', ' mountains', '\n\n', 'The']
[' including', ' interest', ' -', ' to']
[' procedures', ',', ' and', ' thus']
[' because', ' of', ' some', ' “']
[' reported', ' the', ' relationship', '.']
[' wrote', ' to', ' V', '.']


In [8]:
utils.test_prompt("I went to the store and bought some", "_", model)

Tokenized prompt: ['<bos>', 'I', ' went', ' to', ' the', ' store', ' and', ' bought', ' some']
Tokenized answer: [' _']


Performance on answer token:
Rank: 464      Logit: -12.25 Prob:  0.01% Token: | _|

Top 0th token. Logit: -4.16 Prob: 22.56% Token: | items|
Top 1th token. Logit: -5.03 Prob:  9.38% Token: | new|
Top 2th token. Logit: -5.28 Prob:  7.32% Token: | things|
Top 3th token. Logit: -5.38 Prob:  6.64% Token: | groceries|
Top 4th token. Logit: -5.81 Prob:  4.30% Token: | snacks|
Top 5th token. Logit: -6.00 Prob:  3.56% Token: | products|
Top 6th token. Logit: -6.03 Prob:  3.47% Token: | milk|
Top 7th token. Logit: -6.16 Prob:  3.05% Token: | cleaning|
Top 8th token. Logit: -6.22 Prob:  2.87% Token: | stuff|
Top 9th token. Logit: -6.84 Prob:  1.53% Token: | batteries|


Ranks of the answer tokens: [(' _', 464)]